<a href="https://colab.research.google.com/github/ta03-jp/muchu3/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install Flask pyngrok matplotlib

from flask import Flask, jsonify, request
from google.colab import drive
from pyngrok import ngrok
from PIL import Image
import matplotlib.pyplot as plt

app = Flask(__name__)

# Google Driveをマウントする関数
def mount_google_drive(mount_point='/content/drive'):
    """
    Google Colab で Google ドライブを指定したマウントポイントにマウントする関数。

    Args:
        mount_point (str): ドライブをマウントするパス。デフォルトは '/content/drive'。

    Returns:
        str: マウント成功かどうかのメッセージ。
    """
    try:
        drive.mount(mount_point)
        return "Google Drive successfully mounted at " + mount_point
    except Exception as e:
        return str(e)

# ルートエンドポイントの追加
@app.route('/')
def home():
    return "Welcome to the Google Drive Mount API!"

# Google DriveをマウントするAPIエンドポイント
@app.route('/mount_drive', methods=['GET'])
def mount_drive():
    result = mount_google_drive()
    return jsonify({"message": result})

#PNGファイルの読み込み
from PIL import Image
png_path = '/content/drive/MyDrive/ZEMI/VketCloud/template/longsleeve/T_M_0004_000_col.png'

#絶対パスなのでファイルごとにここは変わります！
image = Image.open(png_path)


# PNGファイルの読み込みの関数化
def open_image(png_path):
    """
    指定されたパスのPNG画像ファイルを開き、PIL.Imageオブジェクトを返す関数

    Args:
        png_path (str): 開くPNG画像ファイルのパス

    Returns:
        Image: 開いた画像のPIL.Imageオブジェクト
    """
    try:
        # 画像を開く
        image = Image.open(png_path)
        print(f"画像 {png_path} を正常に開きました。")
        return image
    except Exception as e:
        print(f"画像を開く際にエラーが発生しました: {e}")
        return None

# PNGファイルの読み込みするAPIエンドポイント
@app.route('/open_images', methods=['GET'])
def open_images():
    png_path = request.args.get('png_path')
    if not png_path:
        return jsonify({"message": "PNG file path is required"}), 400
    result = open_image(png_path)
    if result is None:
        return jsonify({"message": "Failed to open image"}), 500
    return jsonify({"message": f"Image {png_path} opened successfully"})

# 特定のフレームを抽出して保存する関数
def extract_frame_from_apng(png_path, frame_index, output_path):
    """
    PNGファイル（APNG）から特定のフレームを抽出して保存する関数

    Args:
        png_path (str): 処理するPNGファイルのパス
        frame_index (int): 抽出したいフレームのインデックス（0から始まる）
        output_path (str): 保存するファイルのパス
    """
    try:
        # PNGファイルを開く
        image = Image.open(png_path)

        # フレームが存在するか確認
        if hasattr(image, 'n_frames') and frame_index < image.n_frames:
            # 特定のフレームを選択
            image.seek(frame_index)
            # フレームを保存
            image.save(output_path)
            return f"Frame {frame_index} saved to {output_path}"
        else:
            return "Specified frame does not exist or file is not an APNG."
    except Exception as e:
        return str(e)

# 特定のフレームを抽出するAPIエンドポイント
@app.route('/extract_frame', methods=['GET'])
def extract_frame():
    png_path = request.args.get('png_path')
    frame_index = request.args.get('frame_index', type=int)
    output_path = request.args.get('output_path')

    if not png_path or frame_index is None or not output_path:
        return jsonify({"message": "png_path, frame_index, and output_path are required"}), 400

    result = extract_frame_from_apng(png_path, frame_index, output_path)
    return jsonify({"message": result})

# 画像を表示する関数
def display_image(png_path):
    """
    指定されたパスのPNG画像ファイルを表示する関数

    Args:
        png_path (str): 表示するPNG画像ファイルのパス
    """
    try:
        # 画像を開く
        image = Image.open(png_path)
        # 画像を表示する
        plt.imshow(image)
        plt.axis('on')  # 軸を表示する
        plt.show()
        return f"Image {png_path} displayed successfully"
    except Exception as e:
        return str(e)

# 画像を表示するAPIエンドポイント
@app.route('/display_image', methods=['GET'])
def display_image_endpoint():
    png_path = request.args.get('png_path')
    if not png_path:
        return jsonify({"message": "PNG file path is required"}), 400

    result = display_image(png_path)
    return jsonify({"message": result})

# ngrokの認証トークンを設定
ngrok.set_auth_token("2lyD3UWDJjzvyARhPUW3dHl4BEB_4D79PdBBWxvymRDTtp6Dj")  # トークンを設定

# ngrokで外部からアクセスできるURLを生成
public_url = ngrok.connect(5000)
print(f"外部アクセス用のURL: {public_url}")

# Flaskサーバーの実行
if __name__ == "__main__":
    app.run(port=5000, threaded=True)


外部アクセス用のURL: NgrokTunnel: "https://79be-35-227-3-143.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
